In [0]:
import cv2
import operator
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [0]:
def pre_process_image(img, skip_dilate=False):
  proc = cv2.GaussianBlur(img.copy(), (9, 9), 0)
  proc = cv2.adaptiveThreshold(proc, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
  proc = cv2.bitwise_not(proc, proc)
  return proc
 

In [0]:
def find_corners(img):
  _,contours,h =  cv2.findContours(img.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  contours = sorted(contours,key = cv2.contourArea,reverse= True)
  polygon = contours[0]
  bottom_right, _ = max(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
  top_left, _ = min(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
  bottom_left, _ = min(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
  top_right, _ = max(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
  return [polygon[top_left][0],polygon[top_right][0],polygon[bottom_right][0],polygon[bottom_left][0]]

In [0]:
def display_points(in_img, points, radius=5, colour=(0, 0, 255)):
	"""Draws circular points on an image."""
	img = in_img.copy()

	# Dynamically change to a colour image if necessary
	if len(colour) == 3:
		if len(img.shape) == 2:
			img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
		elif img.shape[2] == 1:
			img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

	for point in points:
		img = cv2.circle(img, tuple(int(x) for x in point), radius, colour, -1)
	cv2_imshow(img)
	return img

In [0]:
def distance_between(p1,p2):
  a= p2[0]-p1[0]
  b= p2[1]-p1[1]
  return np.sqrt((a*a) + (b*b))

In [0]:
def crop_and_warp(img,crop_rect):
  top_left, top_right, bottom_right, bottom_left = crop_rect[0], crop_rect[1], crop_rect[2], crop_rect[3]
  src = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')
  side = max([
		distance_between(bottom_right, top_right),
		distance_between(top_left, bottom_left),
		distance_between(bottom_right, bottom_left),
		distance_between(top_left, top_right)
	])
  dst = np.array([[0, 0], [side - 1, 0], [side - 1, side - 1], [0, side - 1]], dtype='float32')
  m = cv2.getPerspectiveTransform(src, dst)
  return cv2.warpPerspective(img, m, (int(side), int(side)))

In [0]:
def infer_grid(img):
  squares= []
  side= img.shape[:1]
  side= side[0]/9
  for i in range(9):
    for j in range(9):
      p1= (i*side, j*side)
      p2= ((i+1)*side ,(j+1)*side)
      squares.append((p1,p2))
  return squares

In [0]:
def display_rects(in_img, rects, colour=255):
	"""Displays rectangles on the image."""
	img = in_img.copy()
	for rect in rects:
		img = cv2.rectangle(img, tuple(int(x) for x in rect[0]), tuple(int(x) for x in rect[1]), colour)
	cv2_imshow(img)
	return img

In [0]:
def cut_from_rect(img, rect):
  return img[int(rect[0][1]):int(rect[1][1]), int(rect[0][0]):int(rect[1][0])]

In [0]:
def extract_digit(img, rect, size):
  digit = cut_from_rect(img, rect)  # Get the digit box from the whole square

  h, w = digit.shape[:2]
  margin = int(np.mean([h, w]) / 2.5)
  _, bbox, seed = find_largest_feature(digit, [margin, margin], [w - margin, h - margin])
  digit = cut_from_rect(digit, bbox)

	# Scale and pad the digit so that it fits a square of the digit size we're using for machine learning
  w = bbox[1][0] - bbox[0][0]
  h = bbox[1][1] - bbox[0][1]

	# Ignore any small bounding boxes
  if w > 0 and h > 0 and (w * h) > 100 and len(digit) > 0:
    return scale_and_centre(digit, size, 4)
  else:
    return np.zeros((size, size), np.uint8)



In [0]:
def get_digits(img, squares, size):
  digits = []
  img = pre_process_image(img.copy(), skip_dilate=True)
  for square in squares:
    digits.append(extract_digit(img, square, size))
  return digits

In [0]:
def find_largest_feature(inp_img, scan_tl=None, scan_br=None):
  img = inp_img.copy()  # Copy the image, leaving the original untouched
  height, width = img.shape[:2]
  max_area = 0
  seed_point = (None, None)

  if scan_tl is None:
	  scan_tl = [0, 0]

  if scan_br is None:
	  scan_br = [width, height]

  for x in range(scan_tl[0], scan_br[0]):
	  for y in range(scan_tl[1], scan_br[1]):
		  if img.item(y, x) == 255 and x < width and y < height:  # Note that .item() appears to take input as y, x
			  area = cv2.floodFill(img, None, (x, y), 64)
			  if area[0] > max_area:  # Gets the maximum bound area which should be the grid
				  max_area = area[0]
				  seed_point = (x, y)

	# Colour everything grey (compensates for features outside of our middle scanning range
  for x in range(width):
    for y in range(height):
      if img.item(y, x) == 255 and x < width and y < height:
        cv2.floodFill(img, None, (x, y), 64)

  mask = np.zeros((height + 2, width + 2), np.uint8)  # Mask that is 2 pixels bigger than the image

	# Highlight the main feature
  if all([p is not None for p in seed_point]):
    cv2.floodFill(img, mask, seed_point, 255)

  top, bottom, left, right = height, 0, width, 0

  for x in range(width):
    for y in range(height):
      if img.item(y, x) == 64:  # Hide anything that isn't the main feature
        cv2.floodFill(img, mask, (x, y), 0)

			# Find the bounding parameters
      if img.item(y, x) == 255:
        top = y if y < top else top
        bottom = y if y > bottom else bottom
        left = x if x < left else left
        right = x if x > right else right

  bbox = [[left, top], [right, bottom]]
  return img, np.array(bbox, dtype='float32'), seed_point


In [0]:
def scale_and_centre(img, size, margin=0, background=0):
  h, w = img.shape[:2]

  def centre_pad(length):
	  if length % 2 == 0:
		  side1 = int((size - length) / 2)
		  side2 = side1
	  else:
		  side1 = int((size - length) / 2)
		  side2 = side1 + 1
	  return side1, side2
  def scale(r, x):
	  return int(r * x)

  if h > w:
	  t_pad = int(margin / 2)
	  b_pad = t_pad
	  ratio = (size - margin) / h
	  w, h = scale(ratio, w), scale(ratio, h)
	  l_pad, r_pad = centre_pad(w)
  else:
	  l_pad = int(margin / 2)
	  r_pad = l_pad
	  ratio = (size - margin) / w
	  w, h = scale(ratio, w), scale(ratio, h)
	  t_pad, b_pad = centre_pad(h)

  img = cv2.resize(img, (w, h))
  img = cv2.copyMakeBorder(img, t_pad, b_pad, l_pad, r_pad, cv2.BORDER_CONSTANT, None, background)
  return cv2.resize(img, (size, size))


In [0]:
def show_digits(digits, colour=255):
	"""Shows list of 81 extracted digits in a grid format"""
	rows = []
	with_border = [cv2.copyMakeBorder(img.copy(), 1, 1, 1, 1, cv2.BORDER_CONSTANT, None, colour) for img in digits]
	for i in range(9):
		row = np.concatenate(with_border[i * 9:((i + 1) * 9)], axis=1)
		rows.append(row)
	cv2_imshow(np.concatenate(rows))


In [0]:
def load_model():
  kr_model=tf.keras.models.load_model('final.pt')
  kr_model.compile(optimizer = tf.train.AdamOptimizer(),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return kr_model

In [0]:
def findNextCellToFill(grid, i, j):
        for x in range(i,9):
                for y in range(j,9):
                        if grid[x][y] == 0:
                                return x,y
        for x in range(0,9):
                for y in range(0,9):
                        if grid[x][y] == 0:
                                return x,y
        return -1,-1

def isValid(grid, i, j, e):
        rowOk = all([e != grid[i][x] for x in range(9)])
        if rowOk:
                columnOk = all([e != grid[x][j] for x in range(9)])
                if columnOk:
                        # finding the top left x,y co-ordinates of the section containing the i,j cell
                        secTopX, secTopY = 3 *(i//3), 3 *(j//3) #floored quotient should be used here. 
                        for x in range(secTopX, secTopX+3):
                                for y in range(secTopY, secTopY+3):
                                        if grid[x][y] == e:
                                                return False
                        return True
        return False

def solveSudoku(grid, i=0, j=0):
        i,j = findNextCellToFill(grid, i, j)
        if i == -1:
                return True
        for e in range(1,10):
                if isValid(grid,i,j,e):
                        grid[i][j] = e
                        if solveSudoku(grid, i, j):
                                return True
                        # Undo the current cell for backtracking
                        grid[i][j] = 0
        return False

In [0]:
img = cv2.imread('example5.jpg', cv2.IMREAD_GRAYSCALE)
processed = pre_process_image(img)
corners = find_corners(processed)
#display_points(processed,corners)
cropped = crop_and_warp(img, corners)
#cv2_imshow(cropped)
squares = infer_grid(cropped)
#display_rects(cropped, squares)
digits = get_digits(cropped, squares, 28)
#show_digits(digits)

In [114]:
kr_model=load_model()

W0722 10:39:11.161264 139856666957696 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [0]:
X_train=np.asarray(digits)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
y_pred=kr_model.predict(X_train)
pred=kr_model.predict(X_train)
y_pred = np.argmax(pred, axis=1)
sudoku= y_pred.reshape(9,9).T
grid=sudoku.copy()
flag=solveSudoku(grid)

In [117]:
if flag:
  #cv2_imshow(img)
  print(grid)
else:
  print("no solution exits!!!")

[[5 8 9 7 6 2 4 3 1]
 [3 7 6 4 8 1 9 5 2]
 [4 2 1 5 3 9 7 8 6]
 [9 3 4 2 7 8 6 1 5]
 [7 1 8 6 5 3 2 9 4]
 [2 6 5 1 9 4 3 7 8]
 [6 5 3 8 2 7 1 4 9]
 [1 9 2 3 4 5 8 6 7]
 [8 4 7 9 1 6 5 2 3]]
